In [28]:
import Doc_import
import os
import json
import Data_Preprocessing

In [29]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
train_features, train_labels, test_features, test_labels = Doc_import.import_pan21(base_path=r"C:\Users\jan-p\Documents\Uni Wuppertal\Semester 2\Information "
                                 r"Retrival\Projekt\Data\pan21")


In [63]:

encoded_input = Data_Preprocessing.preprocessing(train_features[0])
encoded_input

{'input_ids': tensor([[  101,  2004,  3357, 16001,  2669,  2056,  1999,  1996,  7928,  2000,
          2115,  3160,  1010,  9207,  8785,  2003,  5186,  2590,  2005,  3492,
          2172,  2151, 14134,  2030,  7605,  2208,  1012,  2174,  1010,  5584,
          3716,  3475,  1005,  1056,  4072,  2005,  1037,  2843,  1997,  3722,
          2399,  1012,  2045,  2024,  5584,  1011,  2066,  8474,  2017,  2323,
          3305,  1037,  2978,  2055,  1010,  2066, 12365,  1010,  2021,  2017,
          2180,  1005,  1056,  2342, 19276,  2030,  5584,  4280,  2005,  2008,
          2004,  2146,  2004,  2017,  2562,  2009,  3722,  1012,  1037,  2843,
          1997,  2477,  2017,  2089,  2215,  2000,  2079,  2064,  2022, 23599,
          3432,  2438,  2008,  2867,  2180,  1005,  1056,  2729,  2172,  1010,
          2066, 15012,  1010,  2030,  8058,  1010,  2030,  8992,  1012,  1037,
         11519, 10616,  1997,  5584,  2097,  3497,  2393,  1999,  2116,  8146,
          2295,  1012,   102],
       

In [64]:
len(encoded_input['input_ids'])

8

In [79]:
len(train_features[0])

"The first time I wanted to have a character jumping in a game, I started using sin(x). The result was kinda akwards: as soon as you jumped off a cliff, you'd end up travelling up and down, as if you'd be riding a curious deltaplane."

In [78]:
train_features[0]

["As stephelton said in the comments to your question, vector math is extremely important for pretty much any 2D or 3D game. However, physics knowledge isn't necessary for a lot of simple games. There are physics-like concepts you should understand a bit about, like collision, but you won't need calculus or physics classes for that as long as you keep it simple. A lot of things you may want to do can be simulated simply enough that players won't care much, like friction, or sliding, or gravity. A decent grasp of physics will likely help in many situations though.\n",
 'It is probably not required to know physics in details when you\'re doing a game, but it definitely helps, especially if there are some \'virtual reality\' features in your game. A game like "From Dust" (Eric Chahi) is essentially physics simulation gamified, while "Another World" only need high-precision capture of real-life motion (so and requires little to no actual understanding of what happens).\n',
 "It is very lik

In [48]:
encoded_input['input_ids']

tensor([[  101,  2004,  3357, 16001,  2669,  2056,  1999,  1996,  7928,  2000,
          2115,  3160,  1010,  9207,  8785,  2003,  5186,  2590,  2005,  3492,
          2172,  2151, 14134,  2030,  7605,  2208,  1012,  2174,  1010,  5584,
          3716,  3475,  1005,  1056,  4072,  2005,  1037,  2843,  1997,  3722,
          2399,  1012,  2045,  2024,  5584,  1011,  2066,  8474,  2017,  2323,
          3305,  1037,  2978,  2055,  1010,  2066, 12365,  1010,  2021,  2017,
          2180,  1005,  1056,  2342, 19276,  2030,  5584,  4280,  2005,  2008,
          2004,  2146,  2004,  2017,  2562,  2009,  3722,  1012,  1037,  2843,
          1997,  2477,  2017,  2089,  2215,  2000,  2079,  2064,  2022, 23599,
          3432,  2438,  2008,  2867,  2180,  1005,  1056,  2729,  2172,  1010,
          2066, 15012,  1010,  2030,  8058,  1010,  2030,  8992,  1012,  1037,
         11519, 10616,  1997,  5584,  2097,  3497,  2393,  1999,  2116,  8146,
          2295,  1012,   102],
        [  101,  2009

In [62]:
len(encoded_input['input_ids'])

8

In [50]:

output = model(**encoded_input)


In [83]:
print(len(output))
#output[0]

2


In [85]:
len(output[0])

8

In [7]:
train_labels

[{'authors': 3,
  'structure': [12611, 5862, 1424, 5862],
  'site': 'gamedev.stackexchange.com.7z',
  'multi-author': 1,
  'changes': [1, 0, 0, 0, 1, 1, 0],
  'paragraph-authors': [1, 2, 2, 2, 2, 3, 2, 2]},
 {'authors': 2,
  'structure': [2062, 2310, 2062],
  'site': 'networkengineering.stackexchange.com.7z',
  'multi-author': 1,
  'changes': [0, 0, 1, 1],
  'paragraph-authors': [1, 1, 1, 2, 1]},
 {'authors': 4,
  'structure': [13726, 19152, 13726, 6436, 19152, 2477, 6436, 2477],
  'site': 'serverfault.com.7z',
  'multi-author': 1,
  'changes': [1, 1, 1, 0, 0, 1, 1, 1, 1],
  'paragraph-authors': [1, 2, 1, 3, 3, 3, 2, 4, 3, 4]},
 {'authors': 4,
  'structure': [9278, 55514, 15677, 55514, 9278, 8625, 55514, 15677],
  'site': 'serverfault.com.7z',
  'multi-author': 1,
  'changes': [1, 1, 1, 0, 1, 1, 1, 0, 1],
  'paragraph-authors': [1, 2, 3, 2, 2, 1, 4, 2, 2, 3]},
 {'authors': 3,
  'structure': [9110, 10193, 9110, 10193, 9110, 4131, 10193, 9110],
  'site': 'serverfault.com.7z',
  'multi-au

In [ ]:
base_path = r"C:\Users\jan-p\Documents\Uni Wuppertal\Semester 2\Information Retrival\Projekt\Data\pan21"

test_train = os.listdir(base_path)

train_features = []
train_labels = []
test_features = []
test_labels = []

for dic in test_train:
     path = base_path + "/" + dic
     data_path = os.listdir(path)

     if dic == test_train[0]:
          for i, file_name in enumerate(data_path[0:int(len(data_path)/2)]):
               file_path = path + "/" + file_name
               with open(file_path, 'r', encoding='utf8') as f:
                    text = f.readlines()

                    train_features.append(text)

          for i, file_name in enumerate(data_path[int(len(data_path)/2):int(len(data_path))]):
               file_path = path + "/" + file_name
               f = open(file_path)
               text = json.load(f)
               train_labels.append(text)

     if dic == test_train[1]:
          for i, file_name in enumerate(data_path[0:int(len(data_path)/2)]):
               file_path = path + "/" + file_name
               with open(file_path, 'r', encoding='utf8') as f:
                    text = f.readlines()

                    test_features.append(text)

          for i, file_name in enumerate(data_path[int(len(data_path)/2):int(len(data_path))]):
               file_path = path + "/" + file_name
               f = open(file_path)
               text = json.load(f)
               train_labels.append(text)




In [55]:
train_labels

[{'authors': 3,
  'structure': [12611, 5862, 1424, 5862],
  'site': 'gamedev.stackexchange.com.7z',
  'multi-author': 1,
  'changes': [1, 0, 0, 0, 1, 1, 0],
  'paragraph-authors': [1, 2, 2, 2, 2, 3, 2, 2]},
 {'authors': 2,
  'structure': [2062, 2310, 2062],
  'site': 'networkengineering.stackexchange.com.7z',
  'multi-author': 1,
  'changes': [0, 0, 1, 1],
  'paragraph-authors': [1, 1, 1, 2, 1]},
 {'authors': 4,
  'structure': [13726, 19152, 13726, 6436, 19152, 2477, 6436, 2477],
  'site': 'serverfault.com.7z',
  'multi-author': 1,
  'changes': [1, 1, 1, 0, 0, 1, 1, 1, 1],
  'paragraph-authors': [1, 2, 1, 3, 3, 3, 2, 4, 3, 4]},
 {'authors': 4,
  'structure': [9278, 55514, 15677, 55514, 9278, 8625, 55514, 15677],
  'site': 'serverfault.com.7z',
  'multi-author': 1,
  'changes': [1, 1, 1, 0, 1, 1, 1, 0, 1],
  'paragraph-authors': [1, 2, 3, 2, 2, 1, 4, 2, 2, 3]},
 {'authors': 3,
  'structure': [9110, 10193, 9110, 10193, 9110, 4131, 10193, 9110],
  'site': 'serverfault.com.7z',
  'multi-au

In [43]:
len(train_features)

13600